# Strands Agents と AgentCore Memory（長期記憶）- ツールを使用した実装

## 概要
このノートブックでは、Strands と AgentCore Memory を使用して会話型 AI エージェントに長期記憶機能を実装する方法を紹介します。短期的なインタラクションから重要な情報を抽出・統合し、エージェントが複数の会話セッションにわたって重要な詳細を想起できるようにする方法を学びます。

## チュートリアルの詳細
**ユースケース:** 永続的な記憶を持つ料理アシスタント

| 項目               | 詳細                                                                             |
|:-------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ | 長期記憶型会話                                                                   |
| エージェントタイプ   | 料理アシスタント                                                                 |
| エージェントフレームワーク | Strands Agents                                                              |
| LLM モデル          | Anthropic Claude Haiku 4.5                                                      |
| チュートリアルコンポーネント | AgentCore 'User Preferences' Memory 抽出、記憶の保存と取得のための Memory ツール |
| 難易度              | 初級                                                                             |

学習内容：
- 長期保持のための抽出戦略を使用した AgentCore Memory の設定
- 過去の会話履歴を使ったメモリのハイドレーション
- 長期記憶を活用して会話セッション間でパーソナライズされた体験を提供
- Strands Agent フレームワークと AgentCore Memory ツールの統合

## シナリオの背景

このチュートリアルでは、高度にパーソナライズされたレストラン推薦を提供する料理アシスタントの役割を体験します。AgentCore Memory の長期保持機能と自動情報抽出を活用することで、エージェントは食事の好みやお気に入りの料理などのユーザー設定を複数の会話にわたって記憶できます。この永続的な記憶により、エージェントは日や週をまたぐ会話においても、カスタマイズされた提案とシームレスなユーザー体験を提供できます。このシナリオは、構造化された記憶の整理と設定可能な戦略が、会話型 AI を短期的な想起を超えて、真に魅力的でコンテキストを認識したインタラクションを生み出す方法を示しています。


## アーキテクチャ

<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>


## 前提条件

このチュートリアルを実行するには以下が必要です：
- Python 3.10以上
- Amazon Bedrock AgentCore Memory 権限を持つ AWS 認証情報
- Amazon Bedrock AgentCore SDK

環境をセットアップし、適切な抽出戦略を持つ長期記憶リソースを作成することから始めましょう！

## ステップ 1: 環境のセットアップ
まず、このノートブックを動作させるために必要なすべてのライブラリをインポートし、クライアントを定義します。

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import time
import logging
import time
from datetime import datetime

Amazon Bedrock モデルと AgentCore に対する適切な権限を持つリージョンとロールを定義します。

In [ ]:
import os

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("culinary-memory")

region = os.getenv('AWS_REGION', 'us-west-2')

## ステップ 2: 長期記憶戦略を使用したメモリの作成

このセクションでは、長期記憶機能を設定したメモリリソースを作成します。以前の短期記憶の例とは異なり、この実装には統合された情報保持を可能にする特定のメモリ戦略が含まれています。

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

client = MemoryClient(region_name=region)

memory_name = "CulinaryAssistant"
memory_id = None

In [ ]:
from botocore.exceptions import ClientError

try:
    print("Creating Long-Term Memory...")

    # We use a more descriptive name for our long-term memory resource
    memory_name = memory_name

    # Create memory with user preference strategy
    memory = client.create_memory_and_wait(
        name=memory_name,
        description="Culinary Assistant Agent with long term memory",
        strategies=[{
                    StrategyType.USER_PREFERENCE.value: {
                        "name": "UserPreferences",
                        "description": "Captures user preferences",
                        "namespaces": ["user/{actorId}/preferences"]
                    }
                }],
        event_expiry_days=7,
        max_wait=300,
        poll_interval=10
    )

    memory_id = memory['id']
    print(f"Memory created successfully with ID: {memory_id}")
    
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

### 長期記憶戦略の理解

このメモリ作成における主な違いは、**メモリ戦略**の追加です。コンポーネントを詳しく見てみましょう：

#### 1. User Preference Memory 戦略

この戦略は、会話からユーザーの好みを自動的に識別し抽出します：

```python
"userPreferenceMemoryStrategy": {
    "name": "UserPreferences",
    "description": "Captures user preferences",
    "namespaces": ["user/{actorId}/preferences"]
}
```

#### 2. メモリ名前空間

`namespaces` パラメータは、抽出された情報の保存場所を定義します：

```python
"namespaces": ["user/{actorId}/preferences"]
```

このメモリ戦略は、単に会話を記憶するだけでなく、会話内の重要な情報を実際に理解し、将来の使用のために整理するより洗練されたメモリシステムを作成します。

## ステップ 3: 過去の会話をメモリに保存

このセクションでは、短期記憶をハイドレーションする方法を紹介します。これにより、バックグラウンドで長期記憶抽出プロセスが自動的にトリガーされます。

### 短期記憶のハイドレーション

抽出戦略で設定されたメモリリソースに会話を保存すると、システムは追加のコードを必要とせずに、この情報を長期保持用に自動的に処理します。

In [ ]:
actor_id = f"user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"foodie-{datetime.now().strftime('%Y%m%d%H%M%S')}"
namespace = f"user/{actor_id}/preferences"

In [ ]:
previous_messages = [
    ("Hi, I'm John", "USER"),
    ("Hi John, how can I help you with food recommendations today?", "ASSISTANT"),
    ("I'm looking for some vegetarian dishes to try this weekend.", "USER"),
    ("That sounds great! I'd be happy to help with vegetarian recommendations. Do you have any specific ingredients or cuisine types you prefer?", "ASSISTANT"),
    ("Yes, I really like tofu and fresh vegetables in my dishes", "USER"),
    ("Perfect! Tofu and fresh vegetables make for excellent vegetarian meals. I can suggest some stir-fries, Buddha bowls, or tofu curries. Do you have any other preferences?", "ASSISTANT"),
    ("I also really enjoy Italian cuisine. I love pasta dishes and would like them to be vegetarian-friendly.", "USER"),
    ("Excellent! Italian cuisine has wonderful vegetarian options. I can recommend pasta primavera, mushroom risotto, eggplant parmesan, or penne arrabbiata. The combination of Italian flavors with vegetarian ingredients creates delicious meals!", "ASSISTANT"),
    ("I spent 2 hours looking through cookbooks but couldn't find inspiring vegetarian Italian recipes", "USER"),
    ("I'm sorry you had trouble finding inspiring recipes! Let me help you with some creative vegetarian Italian dishes. How about stuffed bell peppers with Italian herbs and rice, spinach and ricotta cannelloni, or a Mediterranean vegetable lasagna?", "ASSISTANT"),
    ("Hey, I appreciate food assistants with good taste", "USER"),
    ("Ha! I definitely try to bring good taste to the table! Speaking of which, shall we explore some more vegetarian Italian recipes that might inspire you?", "ASSISTANT")
]

In [ ]:
print("\nHydrating short term memory with previous conversations...")

# Save the conversation history to short-term memory
initial = client.create_event(
    memory_id=memory_id,
    actor_id=actor_id,
    session_id=session_id,
    messages=previous_messages,
)
print("✓ Conversation saved in short term memory")

会話メッセージを含むイベントが正しく保存されたことを確認しましょう。

In [ ]:
events = client.list_events(
    memory_id=memory_id,
    actor_id=actor_id,
    session_id=session_id,
    max_results=5
)
events

このセルはログシステムを設定し、実行中に情報メッセージを表示することで、コードの動作を追跡できるようにします。

### バックグラウンドで何が起こるか

`create_event` の呼び出し後、以下が自動的に実行されます：

1. **短期記憶への保存**: 完全な会話が生の形式で保存されます
2. **抽出のトリガー**: メモリシステムが UserPreference 戦略が設定されていることを検出します
3. **バックグラウンド処理**: 追加のコードなしに、システムは以下を実行します：
   - 会話を分析してプリファレンスの指標を特定
   - 「I'm vegetarian」や「I really enjoy Italian cuisine」などの発言を識別
   - これらのプリファレンスを構造化データとして抽出
4. **長期記憶への統合**: 抽出されたプリファレンスは設定された名前空間（`user/{actorId}/preferences`）に保存されます

抽出と統合は自動的に行われます - エージェントとの会話を維持するか短期記憶をハイドレーションするだけで、メモリ作成時に設定した戦略が残りの処理を行います。

この自動プロセスにより、短期的な会話記録が期限切れになっても、重要な情報が長期記憶に保持されます。


## 長期記憶の取得

このセクションでは、長期記憶に保存された抽出済みプリファレンスにアクセスする方法を探ります。会話のターンに焦点を当てる短期記憶の取得とは異なり、長期記憶の取得は抽出・統合された構造化情報へのアクセスに焦点を当てています。

### 長期記憶からのユーザープリファレンスへのアクセス

長期記憶から情報を取得するには、メモリ作成時に定義した名前空間構造を使用します：


In [ ]:
# Adding a 30s wait to ensure the memory extraction has time to process the event
time.sleep(30)

try:
    # Query the memory system for food preferences
    food_preferences = client.retrieve_memories(
        memory_id=memory_id,
        namespace=namespace,
        query="food preferences",
        top_k=3  # Return up to 3 most relevant results
    )

    if food_preferences:
        print(f"Retrieved {len(food_preferences)} relevant preference records:")
        for i, record in enumerate(food_preferences):
            print(f"\nMemory {i+1}:")
            print(f"- Content: {record.get('content', 'Not specified')}")
    else:
        print("No matching preference records found.")

except Exception as e:
    print(f"Error retrieving preference records: {e}")

このメソッドにより、必要な時に関連する記憶を取得できます。基本を学んだので、エージェントを構築しましょう！

## ステップ 4: エージェントの作成
このセクションでは、ネイティブの `agent_core_memory` ツールを使用して AgentCore Memory を Strands Agent と統合する方法を探ります。

#### 長期記憶機能を持つエージェントのセットアップ
メモリ対応エージェントを作成するには、Strands フレームワークを使用して AgentCore Memory リソースに接続します。

In [ ]:
from strands import tool, Agent
from strands_tools.agent_core_memory import AgentCoreMemoryToolProvider

In [ ]:
system_prompt = f"""あなたは Culinary Assistant（料理アシスタント）、洗練されたレストラン推薦アシスタントです。

目的:
- ユーザーの好みに基づいてレストランを発見する手助けをする
- 会話を通じてユーザーの好みを記憶する
- パーソナライズされたダイニングの推薦を提供する

あなたは以下を可能にする Memory ツールにアクセスできます：
- ユーザーの好み（食事制限、お気に入りの料理、予算の好みなど）を保存
- 以前に保存された情報を取得して推薦をパーソナライズ

"""

In [ ]:
provider = AgentCoreMemoryToolProvider(
    memory_id=memory_id,
    actor_id=actor_id,
    session_id=session_id,
    namespace=namespace
)

agent = Agent(tools=provider.tools, model="global.anthropic.claude-haiku-4-5-20251001-v1:0",system_prompt=system_prompt)

短期記憶と長期記憶の両方にすでにデータを入力したので、エージェントから直接メモリを取得しましょう！

In [ ]:
agent("Give me restaurant recommendations in Irvine based on my food preferences")

エージェントは retrieve_memory_records メソッドを使用してユーザーの記憶を取得したはずです。

素晴らしい！これで AgentCore Long Term Memory から記憶を取得できる Strands Agent が動作するようになりました！

## クリーンアップ
このノートブックで使用したリソースをクリーンアップするためにメモリを削除しましょう。

In [ ]:
#client.delete_memory_and_wait(
#        memory_id = memory_id,
#        max_wait = 300,
#        poll_interval =10
#)